<img src="./img/logoconvexbrancomini2.png"  align="right"/>

<!--
<img src="./img/logoconvexbrancomini2.png"  align="right"/>
-->
# Resource Allocation Problem

<!--
<img src="./img/logoboxverde.png" align="right"/>
-->
__by [Daniel Cinalli](http://www.cinalli.com.br)__ - DSc Artificial Intelligence

## Capacitated Facility Location - Problem #B



<br/><br/> 
## Notes:

* Coded in Python 3.x
* Using [Anaconda](https://www.anaconda.com/) is recommended
* Run the notebook `online` at [binder](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/drcinalli/Artificial-Intelligence-and-Transformation/master)
<!-- * [nbviewer](https://nbviewer.jupyter.org/) allows you to switch the notebooks "slides" mode-->

<br> </br>
### Table of Contents

- [Problem](#prob)
- [Simplex](#simplex)
- [Random Heuristic](#random)
- [Lowest Shipping Cost (per Client) Heuristic](#lowShip)
- [Greatest Shipping Cost (per Client) Heuristic](#maxShip)
- [Lowest Shipping & Fixed Costs Heuristic](#lowShipFix)
- [Greatest Shipping & Fixed Costs Heuristic](#maxShipFix)

<br>
<br>


<a id='prob'></a>
## Problem #B


<br>
Facilities <br>
$|I| = 100$ 
<br>
<br>
Clients <br>
$|J| = 1000$ 
<br>
<br>
Capacity <br>
$u = 8000$ 



<br> 
<br>


<a id='simplex'></a>
### Simplex (exact)



In [9]:
from itertools import product
from math import sqrt
import gurobipy as gp
from gurobipy import GRB
import time

# Get Clients and Facilities
def getFacilities_Clients(file_list):
    return int(file_list[0]), int(file_list[1])

# Get Facilities Fixed Costs
def getFacilities_STRCapacity_FixedCosts(file_list, num_facilities):
    shift = 2
    capacity = []
    cost = []
    
    #loop to get all i location costs
    for i in range(0,num_facilities*2,2):
        #capacity.append(file_list[i+shift])
        capacity.append(8000)
        cost.append(int(file_list[i+1+shift].replace(".","")))
    
    return capacity, cost


# Get Demand and Allocation Costs for j(customer) to each i(client)
def getClient_Demand_AllocationCosts(file_list, num_facilities, num_customers):
    shift = 2 + (num_facilities*2)
    demand = []
    allocation_cost = []
    
    #loop to get all j Clients 
    j=0
    for r in range(0,num_customers):
    
        #get demand
        demand.append(int(file_list[j+shift]))

        #loop to get all i location costs
        for i in range(0,num_facilities):
            allocation_cost.append(float(file_list[j+1+i+shift]))
            
        #fix j
        j += num_facilities+1
 
    
    return demand, allocation_cost

#Read File from OR datasets
fileName='datasets/ORcapa'
ORlist = []

with open(fileName, "r") as f:
    ORlist = f.read().split()
    
##### Sets and Indices #####
num_facilities, num_customers = getFacilities_Clients(ORlist)
capacity, fixed_cost = getFacilities_STRCapacity_FixedCosts(ORlist, num_facilities)
cartesian_prod = list(product(range(num_customers), range(num_facilities)))
# shipping costs
demand, alloc_cost = getClient_Demand_AllocationCosts(ORlist, num_facilities, num_customers)
shipping_cost = dict(zip(cartesian_prod, alloc_cost))

shipping_demand={}
for k, v in shipping_cost.items():
    shipping_demand[k] = v * demand[k[0]]


In [11]:
    
#setup cost
setup_cost = fixed_cost
#demand of customer
dc = demand
#max production
maxp= capacity

In [26]:
#print(num_facilities)
#print(num_customers)
#print(capacity)
#print(fixed_cost)
#print(cartesian_prod)
#print(demand)
#print(alloc_cost)
#print(shipping_cost)
#print(shipping_demand)


In [27]:


start = time.time()
# MIP  model formulation
m = gp.Model('CFLP')


##### Decision Variable #####
x = m.addVars(num_facilities, vtype=GRB.BINARY, name='x')
y = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='y')

##### Constraints #####
m.addConstrs((y[(c,f)] <= x[f] for c,f in cartesian_prod), name='Shipping')
m.addConstrs((gp.quicksum(y[(c,f)] for f in range(num_facilities)) == 1 for c in range(num_customers)), name='Demand')
m.addConstrs((gp.quicksum(dc[c]*y[(c,f)] for c in range(num_customers)) <= maxp[f]*x[f] for f in range(num_facilities)), name='Capacity')

##### Objective Function #####
#m.setObjective(x.prod(setup_cost)+y.prod(shipping_demand), GRB.MINIMIZE)
m.setObjective(x.prod(setup_cost)+y.prod(shipping_cost), GRB.MINIMIZE)

m.Params.Method = 1
# Options are:-1=automatic, 0=primal simplex, 1=dual simplex, 2=barrier, 3=concurrent, 4=deterministic concurrent, 5=deterministic concurrent simplex

m.optimize()

end = time.time()
print("TIME IS: ",end - start)

Changed value of parameter Method to 1
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 101100 rows, 100100 columns and 400100 nonzeros
Model fingerprint: 0x63f0d508
Variable types: 100000 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+03]
  Objective range  [5e+00, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2.095927e+08
Presolve time: 1.37s
Presolved: 101100 rows, 100100 columns, 400100 nonzeros
Variable types: 100000 continuous, 100 integer (100 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   17609    1.7243870e+07   4.785509e+04   0.000000e+00      5s
   20737    1.7802052e+07   3.874541e+04   0.000000e+00     10s
   22444    1.8020251e+07   1.366857e+05   0.000000e+00     15s
   2

In [1]:
# display optimal values of decision variables

for facility in x.keys():
    if (abs(x[facility].x) > 1e-6):
        print(f"\nBuild a warehouse at location {facility + 1}.")

# Shipments from facilities to customers.

for customer, facility in y.keys():
    if (abs(y[customer, facility].x) > 1e-6):
        print(f"\nClient {customer + 1} receives {round(100*y[customer, facility].x, 2)} % of its demand  from Warehouse {facility + 1} .")

#for v in m.getVars():
#    print(v.varname, v.x)

print(f"\nOptimal total:", m.objVal)

#m.write('UFLP_02_Simplex.lp')

NameError: name 'x' is not defined